In [2]:
import Pkg
Pkg.add(url="https://github.com/lucabrugnolini/VectorAutoregressions.jl")

     Cloning git-repo `https://github.com/lucabrugnolini/VectorAutoregressions.jl`
    Updating git-repo `https://github.com/lucabrugnolini/VectorAutoregressions.jl`
    Updating registry at `C:\Users\matsz\.julia\registries\General.toml`
   Resolving package versions...
   Installed EllipsisNotation ─ v0.4.0
   Installed GrowableArrays ─── v0.1.0
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [dd6ac5f8] + VectorAutoregressions v0.1.0 `https://github.com/lucabrugnolini/VectorAutoregressions.jl#master`
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
⌅ [da5c29d0] + EllipsisNotation v0.4.0
  [7ebbc34c] + GrowableArrays v0.1.0
  [dd6ac5f8] + VectorAutoregressions v0.1.0 `https://github.com/lucabrugnolini/VectorAutoregressions.jl#master`
        Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated -m`
Precompiling project...
  ✓ EllipsisNotation
  ✓ 

In [291]:
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using VectorAutoregressions
using CSV
using Test
using MCMCChains
include("utils.jl")

print_percentiles (generic function with 1 method)

In [295]:
columns = [:Date, :short_us, :real_us, :term_us]
data_df = DataFrame(XLSX.readtable("assets_data_03.24.xlsx", "Macro_Data"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
macro_data = TimeArray(data_df; timestamp = :Date)

209×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 1972-01-01 to 2024-01-01
┌────────────┬────────────┬──────────────┬─────────────┐
│            │ short_us   │ real_us      │ term_us     │
├────────────┼────────────┼──────────────┼─────────────┤
│ 1972-01-01 │     0.0388 │    0.0023571 │   0.0215333 │
│ 1972-04-01 │     0.0455 │   0.00326571 │   0.0159333 │
│ 1972-07-01 │  0.0493333 │   0.00268831 │   0.0135667 │
│ 1972-10-01 │      0.053 │   0.00368448 │   0.0107333 │
│ 1973-01-01 │      0.064 │   0.00337313 │  0.00203333 │
│ 1973-04-01 │  0.0767333 │  -0.00326431 │ -0.00866667 │
│ 1973-07-01 │   0.103533 │   0.00377342 │  -0.0314667 │
│ 1973-10-01 │  0.0923667 │   5.89786e-5 │  -0.0248333 │
│     ⋮      │     ⋮      │      ⋮       │      ⋮      │
│ 2022-07-01 │  0.0282333 │  -0.00602734 │  0.00283333 │
│ 2022-10-01 │  0.0427333 │   0.00696215 │ -0.00443333 │
│ 2023-01-01 │  0.0475333 │   0.00141827 │  -0.0110667 │
│ 2023-04-01 │  0.0513333 │  0.000898898 │     -0.0154 │
│ 2023-07-01

In [296]:
data_df = DataFrame(XLSX.readtable("assets_data_03.24.xlsx", "Indexes"; infer_eltypes=true))[1:end, [:Date, :USA_Net]]
dropmissing!(data_df, disallowmissing=true)
data = TimeArray(data_df; timestamp = :Date)
data = collapse(data, Dates.quarter, last)

dates = timestamp(data) .+ Day(1)

data = TimeArray(data; timestamp = dates)

equity = percentchange(data, :log)

205×1 TimeArray{Float64, 1, Date, Vector{Float64}} 1973-04-01 to 2024-04-01
┌────────────┬────────────┐
│            │ USA_Net    │
├────────────┼────────────┤
│ 1973-04-01 │ -0.0833335 │
│ 1973-07-01 │ -0.0532413 │
│ 1973-10-01 │  0.0402782 │
│ 1974-01-01 │ -0.0900348 │
│ 1974-04-01 │ -0.0428685 │
│ 1974-07-01 │ -0.0874197 │
│ 1974-10-01 │  -0.301121 │
│ 1975-01-01 │  0.0939096 │
│     ⋮      │     ⋮      │
│ 2022-10-01 │ -0.0493364 │
│ 2023-01-01 │  0.0674734 │
│ 2023-04-01 │  0.0731963 │
│ 2023-07-01 │  0.0824719 │
│ 2023-10-01 │ -0.0322941 │
│ 2024-01-01 │   0.111622 │
│ 2024-04-01 │  0.0979943 │
└────────────┴────────────┘
           190 rows omitted

In [297]:
mean(equity) .* 4

1×1 TimeArray{Float64, 1, Date, Vector{Float64}} 2024-04-01 to 2024-04-01
┌────────────┬───────────┐
│            │ USA_Net   │
├────────────┼───────────┤
│ 2024-04-01 │ 0.0917981 │
└────────────┴───────────┘

In [298]:
e_premium = equity .- macro_data[:short_us] ./ 4

204×1 TimeArray{Float64, 1, Date, Vector{Float64}} 1973-04-01 to 2024-01-01
┌────────────┬──────────────────┐
│            │ USA_Net_short_us │
├────────────┼──────────────────┤
│ 1973-04-01 │        -0.102517 │
│ 1973-07-01 │       -0.0791246 │
│ 1973-10-01 │        0.0171865 │
│ 1974-01-01 │        -0.111635 │
│ 1974-04-01 │       -0.0702268 │
│ 1974-07-01 │        -0.117486 │
│ 1974-10-01 │        -0.324538 │
│ 1975-01-01 │        0.0771846 │
│     ⋮      │        ⋮         │
│ 2022-07-01 │         -0.19188 │
│ 2022-10-01 │       -0.0600197 │
│ 2023-01-01 │          0.05559 │
│ 2023-04-01 │         0.060363 │
│ 2023-07-01 │        0.0689052 │
│ 2023-10-01 │       -0.0457858 │
│ 2024-01-01 │        0.0984798 │
└────────────┴──────────────────┘
                 189 rows omitted

In [299]:
mean(e_premium) .* 4

1×1 TimeArray{Float64, 1, Date, Vector{Float64}} 2024-01-01 to 2024-01-01
┌────────────┬──────────────────┐
│            │ USA_Net_short_us │
├────────────┼──────────────────┤
│ 2024-01-01 │        0.0403383 │
└────────────┴──────────────────┘

In [300]:
returns = merge(macro_data, e_premium)

204×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 1973-04-01 to 2024-01-01
┌────────────┬────────────┬──────────────┬─────────────┬──────────────────┐
│            │ short_us   │ real_us      │ term_us     │ USA_Net_short_us │
├────────────┼────────────┼──────────────┼─────────────┼──────────────────┤
│ 1973-04-01 │  0.0767333 │  -0.00326431 │ -0.00866667 │        -0.102517 │
│ 1973-07-01 │   0.103533 │   0.00377342 │  -0.0314667 │       -0.0791246 │
│ 1973-10-01 │  0.0923667 │   5.89786e-5 │  -0.0248333 │        0.0171865 │
│ 1974-01-01 │     0.0864 │  -0.00655872 │  -0.0158667 │        -0.111635 │
│ 1974-04-01 │   0.109433 │ -0.000866128 │      -0.034 │       -0.0702268 │
│ 1974-07-01 │   0.120267 │ -0.000148683 │  -0.0406333 │        -0.117486 │
│ 1974-10-01 │  0.0936667 │  -0.00641213 │  -0.0169667 │        -0.324538 │
│ 1975-01-01 │     0.0669 │  -0.00137404 │      0.0085 │        0.0771846 │
│     ⋮      │     ⋮      │      ⋮       │      ⋮      │        ⋮         │
│ 2022-07-01

In [301]:
returns_summarystats(returns,1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
short_us,0.0388218,0.1752,0.0499883,0.0701083,0.001,0.732581,0.0145833,0.970067,0.306504,1.28764,0.0520833
real_us,0.00869245,0.0357101,0.00267766,0.00830319,-0.0258859,0.0123656,-0.00245872,0.642757,1.23567,0.308044,0.00268779
term_us,0.0156434,0.0351,0.00984224,0.021925,-0.0406333,-0.802433,0.000825,0.882008,0.499251,0.629163,0.0119833
USA_Net_short_us,0.0856362,0.195054,0.0100846,0.0620761,-0.324538,-0.868604,-0.0312701,0.0713791,1.39014,0.117761,0.0231837


In [302]:
CSV.write("usa.csv",returns)

"usa.csv"

In [303]:
model = VAR(values(returns), 1, true)

VAR([0.0767333333333333 -0.0032643068090423034 -0.0086666666666666 -0.10251684479395795; 0.10353333333333299 0.003773416161491721 -0.031466666666666296 -0.07912463252540157; … ; 0.0539666666666666 0.012356876445824887 -0.009566666666666598 -0.045785771432493155; 0.052566666666666595 0.002684271890795644 -0.010966666666666597 0.09847984128758931], [0.10353333333333299 0.0923666666666666 … 0.0539666666666666 0.052566666666666595; 0.003773416161491721 5.897855263142099e-5 … 0.012356876445824887 0.002684271890795644; -0.031466666666666296 -0.02483333333333329 … -0.009566666666666598 -0.010966666666666597; -0.07912463252540157 0.017186488034891395 … -0.045785771432493155 0.09847984128758931], [1.0 1.0 … 1.0 1.0; 0.0767333333333333 0.10353333333333299 … 0.054266666666666595 0.0539666666666666; … ; -0.0086666666666666 -0.031466666666666296 … -0.012766666666666593 -0.009566666666666598; -0.10251684479395795 -0.07912463252540157 … 0.06890518427257061 -0.045785771432493155], [-0.0010656671197905

In [12]:
round.(model.β, digits=6)

4×5 Matrix{Float64}:
 -0.001066   1.01681   -0.229077  0.057198   0.015635
 -0.004058   0.09257    0.450456  0.090799   0.003039
  0.001739  -0.01772    0.128329  0.889771  -0.011675
  0.015426  -0.283758   0.77338   0.702965   0.034438

In [13]:
round.(model.Σ, digits=4)

4×4 Matrix{Float64}:
  0.0001   0.0  -0.0001   0.0001
  0.0      0.0  -0.0     -0.0
 -0.0001  -0.0   0.0001  -0.0
  0.0001  -0.0  -0.0      0.0071

In [14]:
A = [1 2; 3 4]


2×2 Matrix{Int64}:
 1  2
 3  4

In [15]:
B = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [16]:
A * B

2×2 Matrix{Int64}:
  7  10
 15  22

In [17]:
dot(A,B)

30

In [18]:
kron(A,B)

4×4 Matrix{Int64}:
 1   2   2   4
 3   4   6   8
 3   6   4   8
 9  12  12  16

- [ ] Normal Wishart Prior gibs sampler
- [ ] draw posterior parameters
- [ ] describe posterior parameters
- [ ] simulate
- [ ] plot simulation
- [ ] estimate momoments estimation
- [ ] calculate waic i inne miary

In [304]:
test_data = values(returns)[154:end,:]

51×4 Matrix{Float64}:
 0.00286667  -0.00336044    0.0214      -3.20364e-5
 0.00423333   0.00257951    0.0162333   -0.153821
 0.0033      -0.00710024    0.0170667    0.108295
 0.003       -0.00749158    0.0152333    0.118372
 0.00266667  -0.00152444    0.0137667   -0.0329988
 0.00233333   0.000222809   0.0147333    0.0596389
 0.0022      -0.00533607    0.0173      -0.00491949
 0.00196667  -0.00490369    0.018        0.099248
 0.00123333  -0.00346436    0.0258667    0.0251424
 0.00126667   0.00311116    0.0262       0.0540249
 ⋮                                      
 0.00443333  -0.0214703     0.0149667    0.0939515
 0.0137      -0.0258859     0.0156      -0.05795
 0.0282333   -0.00602734    0.00283333  -0.19188
 0.0427333    0.00696215   -0.00443333  -0.0600197
 0.0475333    0.00141827   -0.0110667    0.05559
 0.0513333    0.000898898  -0.0154       0.060363
 0.0542667    0.00484838   -0.0127667    0.0689052
 0.0539667    0.0123569    -0.00956667  -0.0457858
 0.0525667    0.00268427   -

In [270]:
model = VAR(test_data, 1, true)
C_OLS = round.(model.β, digits=4)

4×5 Matrix{Float64}:
 -0.0048   1.2238  -0.2986   0.2416  -0.0187
 -0.0095   0.3539   0.3877   0.308    0.0039
  0.0054  -0.2329   0.1603   0.7074   0.0179
  0.1055  -2.8488   2.556   -3.155   -0.1553

## Normal Wishart Prior gibs sampler
- [x] draw covariance matrix
- [] draw coeff - zrobić, aby działało
- refactor, to ols model, i NormalWishartBVAR
- zrobić testy

In [305]:
include("financial_bvar_temp.jl")

model = FinancialBVAR.NormalWishartBVAR(test_data)

round.(transpose(model.C_OLS), digits=4)

model.C_OLS

5×4 Matrix{Float64}:
 -0.00477869  -0.0094609    0.00535864   0.10549
  1.22379      0.353882    -0.232897    -2.84877
 -0.29864      0.387733     0.160339     2.556
  0.241563     0.307988     0.707393    -3.15503
 -0.0186977    0.00388941   0.017932    -0.155301

In [278]:
FinancialBVAR.sample_posterior!(model, 10000,200)
model.Σ[1,:,:]

4×4 Matrix{Float64}:
  8.34999e-6  -1.243e-6     -2.59811e-6  -5.05592e-5
 -1.243e-6     5.57971e-5   -6.57071e-6  -0.000200666
 -2.59811e-6  -6.57071e-6    6.63328e-6   7.70649e-5
 -5.05592e-5  -0.000200666   7.70649e-5   0.00612377

In [279]:
model.Β

10000×20 Matrix{Float64}:
 -0.00478961  1.20141  -0.388921  …   0.577794  -5.51982   -0.374186
 -0.0025586   1.12852  -0.230233      5.17035   -1.02995   -0.21223
 -0.00450325  1.14627  -0.240585      3.70762   -0.904746  -0.123654
 -0.00641407  1.26213  -0.245466     -0.223619   0.890941  -0.15739
 -0.00366954  1.20758  -0.251822      2.53751   -5.83654   -0.362944
 -0.00498232  1.22985  -0.265632  …   0.943141  -1.68455   -0.137337
 -0.00616851  1.22863  -0.300528      1.78942   -2.38486   -0.333851
 -0.00636337  1.29171  -0.402454      1.60287   -4.61405   -0.130468
 -0.00406473  1.18569  -0.422937      3.56568   -5.01503   -0.0819411
 -0.00415397  1.17349  -0.223824      1.15165   -1.11863   -0.0441608
  ⋮                               ⋱                        
 -0.00451801  1.24633  -0.28962       5.15765   -3.46191   -0.176938
 -0.00172628  1.13456  -0.321215      3.16681   -1.67856    0.196215
 -0.00623901  1.24938  -0.334226      2.30814   -3.76939   -0.213948
 -0.00566002  1.2

In [280]:
describe(model.Β[:,20])

Summary Stats:
Length:         10000
Missing Count:  0
Mean:           -0.155138
Std. Deviation: 0.155041
Minimum:        -0.785657
1st Quartile:   -0.257076
Median:         -0.156115
3rd Quartile:   -0.052497
Maximum:        0.464472
Type:           Float64


In [281]:
mean(model.Β,dims=1)

1×20 Matrix{Float64}:
 -0.00478448  1.22416  -0.298119  0.241667  …  2.54899  -3.16355  -0.155138

In [287]:
chn = Chains(model.Β)

Chains MCMC chain (10000×20×1 Array{Float64, 3}):

Iterations        = 1:1:10000
Number of chains  = 1
Samples per chain = 10000
parameters        = param_1, param_2, param_3, param_4, param_5, param_6, param_7, param_8, param_9, param_10, param_11, param_12, param_13, param_14, param_15, param_16, param_17, param_18, param_19, param_20

Summary Statistics
  parameters      mean       std      mcse     ess_bulk     ess_tail      rhat ⋯
      Symbol   Float64   Float64   Float64      Float64      Float64   Float64 ⋯

     param_1   -0.0048    0.0018    0.0000   10173.6758    9960.6832    0.9999 ⋯
     param_2    1.2242    0.0635    0.0006   10308.4592    9274.9620    1.0000 ⋯
     param_3   -0.2981    0.0638    0.0006   10029.1601   10089.0557    1.0000 ⋯
     param_4    0.2417    0.0912    0.0009   10094.4351    9798.2861    0.9999 ⋯
     param_5   -0.0187    0.0060    0.0001    9192.2453    9565.5919    1.0000 ⋯
     param_6   -0.0095    0.0038    0.0000   10150.7508    9915.2910    0

In [290]:
summarize(chn)


  parameters      mean       std      mcse     ess_bulk     ess_tail      rhat ⋯
      Symbol   Float64   Float64   Float64      Float64      Float64   Float64 ⋯

     param_1   -0.0048    0.0018    0.0000   10173.6758    9960.6832    0.9999 ⋯
     param_2    1.2242    0.0635    0.0006   10308.4592    9274.9620    1.0000 ⋯
     param_3   -0.2981    0.0638    0.0006   10029.1601   10089.0557    1.0000 ⋯
     param_4    0.2417    0.0912    0.0009   10094.4351    9798.2861    0.9999 ⋯
     param_5   -0.0187    0.0060    0.0001    9192.2453    9565.5919    1.0000 ⋯
     param_6   -0.0095    0.0038    0.0000   10150.7508    9915.2910    0.9999 ⋯
     param_7    0.3567    0.1357    0.0014   10095.0307    9759.7069    1.0000 ⋯
     param_8    0.3883    0.1385    0.0014    9822.0323    9920.6357    0.9999 ⋯
     param_9    0.3122    0.1929    0.0019   10186.1684    9929.5051    1.0001 ⋯
    param_10    0.0042    0.0128    0.0001    9263.8234    9998.7550    1.0000 ⋯
    param_11    0.0053    

In [310]:
include("financial_bvar_temp.jl")
model = FinancialBVAR.VARModel(test_data)

Main.FinancialBVAR.VARModel([0.00423333333333332 0.0025795105823884553 0.016233333333333377 -0.1538208820653177; 0.0033 -0.007100235312859051 0.017066666666666598 0.10829454953313074; … ; 0.0539666666666666 0.012356876445824887 -0.009566666666666598 -0.045785771432493155; 0.052566666666666595 0.002684271890795644 -0.010966666666666597 0.09847984128758931], [1.0 0.00286666666666666 … 0.02140000000000004 -3.203636796663648e-5; 1.0 0.00423333333333332 … 0.016233333333333377 -0.1538208820653177; … ; 1.0 0.054266666666666595 … -0.012766666666666593 0.06890518427257061; 1.0 0.0539666666666666 … -0.009566666666666598 -0.045785771432493155], [-0.004778689279757392 -0.009460899455677555 0.0053586412859994706 0.1054899451323232; 1.2237864561013927 0.3538815569760155 -0.23289736413220397 -2.8487737159716464; … ; 0.24156282197569665 0.3079877911536331 0.7073930241177909 -3.1550324945251758; -0.01869765913681341 0.00388941153748372 0.01793200513295851 -0.15530135158803102], [0.0004242587078846737 -

In [312]:
transpose(model.C)

4×5 transpose(::Matrix{Float64}) with eltype Float64:
 -0.00477869   1.22379   -0.29864    0.241563  -0.0186977
 -0.0094609    0.353882   0.387733   0.307988   0.00388941
  0.00535864  -0.232897   0.160339   0.707393   0.017932
  0.10549     -2.84877    2.556     -3.15503   -0.155301

In [316]:
model.Σ

4×4 Matrix{Float64}:
  0.000424259  -0.00013613   -0.000131096   0.000301976
 -0.00013613    0.00192401   -0.000116569  -0.00846495
 -0.000131096  -0.000116569   0.000344468   0.00145968
  0.000301976  -0.00846495    0.00145968    0.27623

In [324]:
include("financial_bvar_temp.jl")
FinancialBVAR.simulate(model, 20)

MethodError: MethodError: no method matching simulate(::Main.FinancialBVAR.VARModel, ::Int64)

Closest candidates are:
  simulate(!Matched::Main.FinancialBVAR.VARModel, ::Int64)
   @ Main.FinancialBVAR c:\Users\matsz\programowanie\Optymalizacja_portfela\julia_msp\financial_bvar_temp.jl:71
